# Converting basal area to carbon stock

**Goal**: Use equations from Torres & Lovett et al., 2013 to convert basal area to carbon stock in the Ecological Reserves of Maine (Kuehne et al.)

Note: Goal not accomplished! This dataset does not include Avg DBH but we went ahead with the excercise anyway - maybe we can use this somewhere else.

## Set UP

In [2]:
import pandas as pd
import os
import math

In [3]:
os.getcwd()

'/Users/emilyelizabeth/Python/Forest-carbon-estimation-main'

## Load data

In [5]:
#load in csv
tree_df = pd.read_csv('ERM_Round1.csv')
tree_df.describe()

,Transect,Plot,Latitude,Longitude,Elevation,Slope,Aspect,Forest_Type,Harvest_History,Age_DBH,Live_Basal_Area,Live_TPH,Large_Live_TPH,Very_Large_Live_TPH,Standing_Dead_TPH,Large_Standing_Dead_TPH,Very_Large_Standing_Dead_TPH,DCWD_Volume,Large_DCWD_Volume
count,1103.000000,1103.000000,1092.000000,1092.000000,1092.000000,1095.000000,1075.000000,1102.000000,1103.000000,1019.000000,1103.000000,1103.000000,1103.000000,1103.000000,1103.000000,1103.000000,1103.000000,1103.000000,1103.000000
mean,11.425204,3.796917,45.266778,-69.254898,301.141941,15.523973,146.656744,353.212341,1.332729,80.505397,24.036016,528.211728,26.523736,4.763495,90.868483,2.748530,0.474559,41.305192,4.086365
std,16.191055,3.927024,0.861314,0.941148,219.310964,19.170286,115.786246,317.242931,1.864485,41.519195,15.460545,320.443549,44.477126,10.740847,106.156767,13.025190,2.567088,50.492386,23.740894
min,1.000000,1.000000,43.228602,-71.022860,5.000000,0.000000,0.000000,100.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,2.000000,44.651433,-69.980603,112.000000,4.000000,30.000000,120.000000,0.000000,51.500000,12.961005,297.418325,0.000000,0.000000,0.000000,0.000000,0.000000,6.779859,0.000000
50%,5.000000,3.000000,45.524755,-69.104535,295.000000,10.000000,142.000000,120.000000,1.000000,75.000000,23.045094,485.745513,0.000000,0.000000,59.483665,0.000000,0.000000,25.739323,0.000000
75%,12.000000,5.000000,45.838584,-68.834779,412.250000,20.250000,246.000000,800.000000,1.000000,104.500000,32.808575,713.803979,59.483665,9.876194,118.967330,0.000000,0.000000,56.859310,0.000000
max,69.000000,38.000000,46.991734,-67.145093,1095.000000,220.000000,360.000000,960.000000,5.000000,300.000000,109.615291,1962.960942,307.294519,98.761940,773.287644,148.595912,39.504776,527.777570,487.648294


Assign community types:
(From Kuehne meta-data)
Forest-type group code according to USDA (2015):
100    White-red-jack pine
120    Spruce-fir
160    Loblolly-shortleaf pine
170    Other eastern softwoods
380    Exotic softwoods
390    Miscellaneous softwoods
400    Oak-pine
500    Oak-hickory
700    Elm-ash-cottonwood
800    Maple-beech-birch
900    Aspen-birch
960    Minor hardwoods

In [6]:
# Assign community types: Con for conifer, TH for Temperate Eastern US hardwood, M for mixed

community_type_dict = {100: 'Con', 120: 'Con', 160: 'Con', 170: 'Con', 380: 'Con', 390: 'Con', 400: 'M', 
                  500: 'TH', 700: 'TH', 800: 'TH', 900: 'TH', 960: 'TH'}

In [7]:
tree_df['comm_key'] = tree_df['Forest_Type'].map(community_type_dict)
tree_df.head()

,Eco_Reserve,Location,Transect,Plot,Latitude,Longitude,Elevation,Slope,Aspect,Forest_Type,...,Live_Basal_Area,Live_TPH,Large_Live_TPH,Very_Large_Live_TPH,Standing_Dead_TPH,Large_Standing_Dead_TPH,Very_Large_Standing_Dead_TPH,DCWD_Volume,Large_DCWD_Volume,comm_key
0,Bigelow,NaN,5,5,45.127134,-70.318425,538.0,12.0,280.0,120.0,...,48.155652,892.254974,0.000000,0.000000,0.000000,0.0,0.0,100.551750,0.0,Con
1,Bigelow,NaN,2,6,45.161601,-70.307540,535.0,22.0,326.0,120.0,...,45.489154,1011.222304,69.359859,9.876194,59.483665,0.0,0.0,15.223659,0.0,Con
2,Bigelow,NaN,2,1,45.171023,-70.299127,427.0,5.0,26.0,800.0,...,39.406938,475.869319,118.967330,0.000000,0.000000,0.0,0.0,29.347640,0.0,TH
3,Salmon Brook Lake,NaN,1,1,46.905978,-68.249843,211.0,0.0,0.0,120.0,...,71.271710,1606.058953,0.000000,0.000000,416.385654,0.0,0.0,84.984442,0.0,Con
4,Salmon Brook Lake,NaN,1,2,46.903759,-68.249007,212.0,0.0,0.0,120.0,...,52.670561,1427.607958,59.483665,0.000000,178.450995,0.0,0.0,65.636993,0.0,Con


Ideally we can add in ways to convert the standing dead wood and coarse down woody debris data as it can contribute up to 30% of total aboveground carbon in some older forests (wow!)

# Equations to convert basal area to biomass & carbon

equation for conifers:
0.5/1000 * k * exp(-1.170) * Dmean^0.119 x Gmean = tons C * ha^-1

equation for temperate hardwoods:
0.5/1000 * k * ((0.5/Davg) + ((25000 * Davg^2.5)/(Davg^4.5 + 246872* Davg^2))) * G

where k is 40,000 * pi^-1
Davg is average DBH per plot
G is basal area in m^2 ha^-1

In [8]:
# a function to convert conifer basal area to carbon stock
def conifer_carbon(D, G):
    k = 40000/math.pi
    carb = 0.5/1000 * k * math.exp(-1.170) * D**0.119 * G
    return carb

#test with the average numbers from the dataset
print(conifer_carbon(80,24))

79.87935129963944


In [9]:
# a function to convert temperate hardwood basal area to carbon stock
def  hardwood_carbon(D, G):
    k = 40000/math.pi
    carb = 0.5/1000 * k * ((0.5/D) + ((25000 * D**2.5)/(D**4.5 + 246872 * D**2)))*G
    return carb

# test it with the average numbers from the dataset
print(hardwood_carbon(80,24))

113.29586602453915


Now to apply the equation functions to each row of the dataset

Oh no, I thought we had Avg DBH!

Oh well, I'm just going to follow through with "Age DBH" now

In [10]:
#pulling out the column names and putting them in a list
col_names= []
for col in tree_df.columns:
    col_names.append(col)

col_names.append('carbon')

tree_df = tree_df.reindex(columns = col_names)
tree_df.head()

,Eco_Reserve,Location,Transect,Plot,Latitude,Longitude,Elevation,Slope,Aspect,Forest_Type,...,Live_TPH,Large_Live_TPH,Very_Large_Live_TPH,Standing_Dead_TPH,Large_Standing_Dead_TPH,Very_Large_Standing_Dead_TPH,DCWD_Volume,Large_DCWD_Volume,comm_key,carbon
0,Bigelow,NaN,5,5,45.127134,-70.318425,538.0,12.0,280.0,120.0,...,892.254974,0.000000,0.000000,0.000000,0.0,0.0,100.551750,0.0,Con,NaN
1,Bigelow,NaN,2,6,45.161601,-70.307540,535.0,22.0,326.0,120.0,...,1011.222304,69.359859,9.876194,59.483665,0.0,0.0,15.223659,0.0,Con,NaN
2,Bigelow,NaN,2,1,45.171023,-70.299127,427.0,5.0,26.0,800.0,...,475.869319,118.967330,0.000000,0.000000,0.0,0.0,29.347640,0.0,TH,NaN
3,Salmon Brook Lake,NaN,1,1,46.905978,-68.249843,211.0,0.0,0.0,120.0,...,1606.058953,0.000000,0.000000,416.385654,0.0,0.0,84.984442,0.0,Con,NaN
4,Salmon Brook Lake,NaN,1,2,46.903759,-68.249007,212.0,0.0,0.0,120.0,...,1427.607958,59.483665,0.000000,178.450995,0.0,0.0,65.636993,0.0,Con,NaN


In [11]:
#define conditions using the loc method
tree_df.loc[tree_df.comm_key == 'Con', 'carbon'] = conifer_carbon(tree_df.Age_DBH, tree_df.Live_Basal_Area)
tree_df.loc[tree_df.comm_key == 'TH', 'carbon'] = hardwood_carbon(tree_df.Age_DBH, tree_df.Live_Basal_Area)
tree_df.loc[tree_df.comm_key == 'M', 'carbon'] = 0.5 * conifer_carbon(tree_df.Age_DBH, tree_df.Live_Basal_Area) + 0.5* hardwood_carbon(tree_df.Age_DBH, tree_df.Live_Basal_Area)
tree_df.head()

,Eco_Reserve,Location,Transect,Plot,Latitude,Longitude,Elevation,Slope,Aspect,Forest_Type,...,Live_TPH,Large_Live_TPH,Very_Large_Live_TPH,Standing_Dead_TPH,Large_Standing_Dead_TPH,Very_Large_Standing_Dead_TPH,DCWD_Volume,Large_DCWD_Volume,comm_key,carbon
0,Bigelow,NaN,5,5,45.127134,-70.318425,538.0,12.0,280.0,120.0,...,892.254974,0.000000,0.000000,0.000000,0.0,0.0,100.551750,0.0,Con,155.187627
1,Bigelow,NaN,2,6,45.161601,-70.307540,535.0,22.0,326.0,120.0,...,1011.222304,69.359859,9.876194,59.483665,0.0,0.0,15.223659,0.0,Con,156.906476
2,Bigelow,NaN,2,1,45.171023,-70.299127,427.0,5.0,26.0,800.0,...,475.869319,118.967330,0.000000,0.000000,0.0,0.0,29.347640,0.0,TH,177.125298
3,Salmon Brook Lake,NaN,1,1,46.905978,-68.249843,211.0,0.0,0.0,120.0,...,1606.058953,0.000000,0.000000,416.385654,0.0,0.0,84.984442,0.0,Con,246.641522
4,Salmon Brook Lake,NaN,1,2,46.903759,-68.249007,212.0,0.0,0.0,120.0,...,1427.607958,59.483665,0.000000,178.450995,0.0,0.0,65.636993,0.0,Con,183.602030


In [12]:
tree_df['carbon'].groupby(tree_df['comm_key']).describe()

,count,mean,std,min,25%,50%,75%,max
comm_key,,,,,,,,
Con,648.0,87.405717,55.716022,0.000000,43.985333,85.727352,118.132466,307.841708
M,53.0,90.851047,53.205792,14.861016,48.669527,85.324264,116.933832,244.472204
TH,310.0,88.005738,60.255706,0.000000,42.029393,83.034140,120.172572,381.051689
